<center><h1><b><span style="color:blue">Particles &amp; decays</span></b></h1></center>

&nbsp;<br>
**Particles and decays are central "concepts" in Particle Physics.** Could one really do without dedicated packages in the ecosystem? ...
Of course not!

### **Quick intro to the following packages**
- [Particle](https://github.com/scikit-hep/particle) - *Particle Data Group* particle data, Monte Carlo identification codes, and more.
- [DecayLanguage](https://github.com/scikit-hep/decaylanguage) - Decay files (notably for EvtGen), universal description of decay chains.

&nbsp;
<center><img src="images/logo_particle.png" alt="Particle package logo" style="width: 150px;"/></center>

<center><h2><b><span style="color:green">PDG particle data, MC identification codes</span></b></h2></center>

**The [Particle Data Group](https://pdg.lbl.gov/) (PDG) is an "international collaboration that provides a comprehensive summary of Particle Physics and related areas of Cosmology: the Review of Particle Physics."**

<center><img src="images/intro_PDG.jpg" style="width:55%"/></center>

The **Review of Particle Physics** is a document whose importance is impossible to oversell.
Some interesting facts about it and its more recent edition [https://pdg.lbl.gov/#about]:

<center><img src="images/intro_PDG_about.png" style="width:55%"/></center>

### Package motivation - particle data

- The PDG provides a series of downloadable <span style="color:green">*Computer Readable Files* and in particular a table of particle masses, widths, etc. and PDG Monte Carlo particle ID numbers</span> (PDG IDs).
The most recent file is [here](http://pdg.lbl.gov/2021/html/computer_read.html).
- It <span style="color:green">also provided an experimental file with extended information</span>
(spin, quark content, P and C parities, etc.) until 2008 only, see [here](http://pdg.lbl.gov/2008/html/computer_read.html) (not widely known!).

- But <span style="color:green"><i>anyone</i> wanting to use these data</span>, the only readily available,
<span style="color:green">has to parse the file programmatically</span>.
- Why not make a Python package to deal with all these data, for everyone?

### Package motivation - MC identification codes

- The <span style="color:green">C++ HepPID and HepPDT libraries provide functions for processing particle ID codes</apan>
in the standard particle (aka PDG) numbering scheme.
- Different event generators may have their separate set of particle IDs: Geant3, etc.
- Again, why not make a package providing all functionality/conversions, Python-ically, for everyone?

### **Pythonic interface to**
- Particle Data Group (PDG) particle data table.
- Particle MC identification codes, with inter-MC converters.
- With various extra goodies.

### Package, in short

- <span style="color:green">Particle</span> - loads extended <b>PDG data tables</b> and implements search and manipulations / display.
- <span style="color:green">PDGID</span> - find out as much as possible from the PDG ID number. <b>No table lookup</b>.
- <span style="color:green">Converters for MC IDs</span> used in [Pythia](https://pythia.org/) and Geant3.
- Flexible / advanced usage programmatically.
- Basic usage via the command line.

### **1. `PDGID` class and MC ID classes**


- Classes `PDGID`, `PythiaID`, `Geant3ID`.
- Converters in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

#### PDG IDs module overview

- <span style="color:green">Process and query PDG IDs</span>, and more – no look-up table needed.
  - Current version of package reflects the latest version of the
    <span style="color:green">HepPID & HepPDT utility functions</span> defined in the C++ HepPID and HepPDT versions 3.04.01
  - It contains more functionality than that available in the C++ code … and minor fixes too.
- Definition of a <span style="color:green">PDGID class, PDG ID literals</span>,
and set of standalone HepPID <span style="color:green">functions to query PDG IDs</span>
(is_meson, has_bottom, j_spin, charge, etc.).
   - All PDGID class functions are available standalone.

#### PDGID class
- Wrapper class `PDGID` for PDG IDs.
- Behaves like an int, with extra goodies.
- Large spectrum of properties and methods, with a Pythonic interface, and yet more!

In [ ]:
from particle import PDGID

In [ ]:
pid = PDGID(211)
pid

In [ ]:
PDGID(99999999)

In [ ]:
from particle.pdgid import is_meson

pid.is_meson, is_meson(pid)

To print all `PDGID` properties:

In [ ]:
print(pid.info())

#### MC ID classes and converters

- <span style="color:green">Classes for MC IDs</span> used in Pythia and Geant3: `PythiaID` and `Geant3ID`.
- <span style="color:green">ID converters</span> in module `particle.converters`: `Geant2PDGIDBiMap`, etc.

In [ ]:
from particle import PythiaID, Geant3ID

pyid = PythiaID(10221)

pyid.to_pdgid()

Conversions are directly available via mapping classes.

E.g., bi-directional map Pythia ID - PDG ID:

In [ ]:
from particle.converters import Pythia2PDGIDBiMap

Pythia2PDGIDBiMap[PDGID(9010221)]

In [ ]:
Pythia2PDGIDBiMap[PythiaID(10221)]

### **2. `Particle` class**

There are various ways to create a particle. The often used method is via its PDG ID.

In [ ]:
from particle import Particle

In [ ]:
Particle.from_pdgid(211)

#### Searching

<span style="color:green">Simple and natural API</span> to deal with the PDG particle data table,<br>with <span style="color:green">powerful 1-line search and look-up utilities!</span>

- `Particle.find(…)` – search a single match (exception raised if multiple particles match the search specifications).
- `Particle.findall(…)` – search a list of candidates.

- Search methods that can query any particle property!

In [ ]:
Particle.find('J/psi')

You can specify search terms as keywords - _any particle property_:

You can directly check the numeric charge:

In [ ]:
Particle.findall('pi', charge=-1)

Or use a **lambda function** for the ultimate in generality! For example, to find all the neutral particles with a bottom quark between 5.2 and 5.3 GeV:

In [ ]:
from hepunits import GeV, s  # Units are good. Use them.

In [ ]:
Particle.findall(lambda p:
                     p.pdgid.has_bottom
                     and p.charge==0
                     and 5.2*GeV < p.mass < 5.3*GeV
                )

Another lambda function example: You can use the width or the lifetime:

In [ ]:
Particle.findall(lambda p: p.lifetime > 1000*s)

If you want infinite lifetime, you could just use the keyword search instead:

In [ ]:
Particle.findall(lifetime=float('inf'))

Trivially find all pseudoscalar charm mesons:

In [ ]:
from particle import SpinType

Particle.findall(lambda p: p.pdgid.is_meson and p.pdgid.has_charm and p.spin_type==SpinType.PseudoScalar)

#### Display

Nice display in Jupyter notebooks, as well as `str` and `repr` support:

In [ ]:
p = Particle.from_pdgid(-415)
p

In [ ]:
print(p)

In [ ]:
print(repr(p))

Full descriptions:

In [ ]:
print(p.describe())

You may find LaTeX or HTML to be more useful in your program; both are supported:

In [ ]:
print(p.latex_name, '\n', p.html_name)

#### Particle properties

You can do things to particles, like **invert** them:

In [ ]:
~p

There are a plethora of properties you can access:

In [ ]:
p.spin_type

You can quickly access the PDGID of a particle:

In [ ]:
p.pdgid

### **3. Literals**

They provide a <span style="color:green">handy way to manipulate things with human-readable names!</span>

`Particle` defines <span style="color:green">literals for all particles</span>, with easily recognisable (programmatic friendly) names.
- Literals are dynamically generated on import for both `PDGID` and `Particle` classes.

#### Particle literals

In [ ]:
from particle import literals as lp

In [ ]:
lp.phi_1020

#### PDGID literals

In [ ]:
from particle.pdgid import literals as lid

In [ ]:
lid.phi_1020

### **4. Data files, stored in `particle/data/`**

- <b>PDG particle data files</b>
  - Original PDG data files, which are in a fixed-width format - simply for bookkeeping and reference.
  - Code rather uses “digested forms” of these, produced within `Particle`, stored as CSV, for optimised querying.
  - Latest PDG data (2020) used by default.
  - Advanced usage: user can load older PDG tables, load a “user table” with new particles, append to default table.

- <b>Other data files</b>
  - CSV file for mapping of PDG IDs to particle LaTeX names.

**Dump table contents**

The package provides the 2 methods `Particle.to_list(...)` and `Particle.to_dict(...)`, which make it easy to dump (selected) particle properties in an easy way. No need to dig into the package installation directory to inspect the particle data table ;-).

Tabular output can be formatted with the powerful package `tabulate`, for example (other similar libraries exist).

In [ ]:
from tabulate import tabulate

query_as_list = Particle.to_list(filter_fn=lambda p: p.pdgid.is_lepton and p.charge!=0, exclusive_fields=['pdgid', 'name', 'mass', 'charge'])

print(tabulate(query_as_list, headers="firstrow"))

Fancy creating tables of particle properties in, say, HTML or reStructuredText format, as below? Then check out the *exercises/particle.ipynb* notebook in the exercises.

<table>
<tr style="background: white;">
    <td align="center"><img src="images/Scikit-HEP_gallery_Particle.jpg" width="80%"></td>
    <td align="center"><img src="images/Scikit-HEP_gallery_Particle_ex-table-rst.png" width="80%"></td>
</tr>
</table>

### **5. Advanced usage**

You can:

* Extend or replace the default particle data table in `Particle`.
* Adjust properties for a particle.
* Make custom particles.

&nbsp;<br>
<center>
    <img src="images/logo_decaylanguage.png" style="width: 200px;"/>
    <h2><b><span style="color:green">Decay files, universal description of decay chains</span></b></h2>
</center>

`DecayLanguage` is designed for the manipulation of decay structures in Python. The current package has:

- Decay file parsers:
  - Read *.dec DecFiles*, such as [EvtGen](https://evtgen.hepforge.org/)<sup>(*)</sup> decay files typically used in Flavour Physics experiments.
  - Manipulate and visualise them in Python.
- Amplitude Analysis decay language:
  - Input based on AmpGen generator, output format for GooFit C++ program.
  
> <sup>(*)</sup> *"EvtGen is a Monte Carlo event generator that simulates the decays of heavy flavour particles, primarily B and D mesons. It contains a range of decay models for intermediate and final states containing scalar, vector and tensor mesons or resonances, as well as leptons, photons and baryons. Decay amplitudes are used to generate each branch of a given full decay tree, taking into account angular and time-dependent correlations which allows for the simulation of CP-violating processes..."*

### Package motivation

- Ability to describe decay-tree-like structures.
- Provide a translation of decay amplitude models from AmpGen to GooFit.
  - Idea is to generalise this to other decay descriptions.
- Any experiment uses event generators which, among many things, need to describe particle decay chains.
- Programs such as EvtGen rely on so-called .dec decay files.
- Many experiments need decay data files.
- Why not make a Python package to deal with decay files, for everyone?

### Package, in short

- Tools to parse decay files and programmatically manipulate them, query, display information.
  - Descriptions and parsing built atop the [Lark parser](https://github.com/lark-parser/lark/).
- Tools to translate decay amplitude models from AmpGen to GooFit, and manipulate them.

### **1. Decay files**

#### *Master file” DECAY.DEC

<span style="color:green">Gigantic file defining decay modes for all relevant particles, including decay model specifications.</span>
The LHCb experiment uses one. Belle II as well, and others.

#### User .dec files
- Needed to produce specific MC samples.
- Typically contain a single decay chain (except if defining inclusive samples).

**Example user decay file:**

<small>
<pre>
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
</pre>
</small>

### **2. Decay file parsing**

- Parsing should be simple
- Parsing should be (reasonably) fast!

After parsing, many queries are possible!

In [ ]:
from decaylanguage import DecFileParser

#### The LHCb "master decay file"

It's a big file! ~ 500 particle decays defined, thousands of decay modes, over 11k lines in total.

In [ ]:
dfp = DecFileParser('data/DECAY_LHCB.DEC')

In [ ]:
%%time
dfp.parse()

In [ ]:
dfp

Let's parse and play with a small decay file:

In [ ]:
with open('data/Dst.dec') as f:
    print(f.read())

In [ ]:
dfp_Dst = DecFileParser('data/Dst.dec')
dfp_Dst

In [ ]:
dfp_Dst.parse()
dfp_Dst

It can be handy to **parse from a multi-line string** rather than a file:

In [ ]:
s = """
# Decay file for [B_c+ -> (B_s0 -> K+ K-) pi+]cc

Alias      B_c+sig        B_c+
Alias      B_c-sig        B_c-
ChargeConj B_c+sig        B_c-sig
Alias      MyB_s0         B_s0
Alias      Myanti-B_s0    anti-B_s0
ChargeConj MyB_s0         Myanti-B_s0

Decay B_c+sig
  1.000     MyB_s0     pi+     PHOTOS PHSP;
Enddecay
CDecay B_c-sig

Decay MyB_s0
    1.000     K+     K-     SSD_CP 20.e12 0.1 1.0 0.04 9.6 -0.8 8.4 -0.6;
Enddecay
CDecay Myanti-B_s0
"""

In [ ]:
dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

#### Decay file information

Several methods are available, e.g.:

In [ ]:
dfp_Dst.list_decay_mother_names()

In [ ]:
dfp_Dst.print_decay_modes('D*+')

#### Info such as particle aliases

In [ ]:
dfp.dict_aliases()

In [ ]:
dfp.dict_charge_conjugates()

### **3.  Display of decay chains**

The parser can provide a simple `dict` representation of any decay chain found in the input decay file(s). Being generic and simple, that is what is used as input information for the viewer class (see below).

In [ ]:
dc = dfp_Dst.build_decay_chains('D+')
dc

In [ ]:
from decaylanguage import DecayChainViewer

In [ ]:
DecayChainViewer(dc)

In [ ]:
dc = dfp_Dst.build_decay_chains('D*+', stable_particles=['D+', 'D0', 'pi0'])
DecayChainViewer(dc)

#### **Charge conjugation**

In [ ]:
dc_cc = dfp_Dst.build_decay_chains('D*-', stable_particles=['D-', 'anti-D0', 'pi0'])
DecayChainViewer(dc_cc)

#### **Parsing several files**

Typically useful when the user decay file needs information from the master decay file.

In [ ]:
s = u"""
Alias      MyXic+              Xi_c+
Alias      MyantiXic-          anti-Xi_c-
ChargeConj MyXic+              MyantiXic-

Decay Xi_cc+sig
  1.000       MyXic+    pi-    pi+       PHSP;
Enddecay
CDecay anti-Xi_cc-sig

Decay MyXic+
  1.000       p+    K-    pi+       PHSP;
Enddecay
CDecay MyantiXic-

End
"""

In [ ]:
dfp = DecFileParser.from_string(s)
dfp.parse()
dfp

Note the subtletly: 3, not 4 decays, are found! This is because the file contains no statement
`ChargeConj anti-Xi_cc-sigXi_cc+sig`, hence the parser cannot know to which particle (matching `Decay` statement) the charge-conjugate decay of `anti-Xi_cc-sig` relates to (code does not rely on position of statements to guess ;-)).

In [ ]:
d = dfp.build_decay_chains('Xi_cc+sig')
DecayChainViewer(d)

As said in the warning, the information provided is not enough for the anti-Xi_cc-sig to make sense:

In [ ]:
from decaylanguage.dec.dec import DecayNotFound

try:
    d = dfp.build_decay_chains('anti-Xi_cc-sig')
except DecayNotFound:
    print("Decays of particle 'anti-Xi_cc-sig' not found in .dec file!")

But the missing information is easily providing **parsing two files simultaneously ...!** (Any number of files is allowed.)

In [ ]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile(delete=False) as tf:
    tf.write(s.encode('utf-8'))
    
dfp = DecFileParser(tf.name, 'data/DECAY_LHCB.DEC')
dfp.parse()

In [ ]:
dc = dfp.build_decay_chains('Xi_cc+sig')

DecayChainViewer(dc)

In [ ]:
dc_cc = dfp.build_decay_chains('anti-Xi_cc-sig')

DecayChainViewer(dc_cc)

Want to save a graph? Try for example 
```python
dcv = DecayChainViewer(...)
dcv.graph.save(...)
```

### **4. Representation of decay chains**

<span style="color:green">The universal (and digital) representation of decay chains is of interest well outside the context of decay file parsing!</span>

#### Building blocks

- A <span style="color:green">daughters list</span> - list of final-state particles.
- A <span style="color:green">decay mode</span> - typically a branching fraction and a list of final-state particles (may also contain _any_ metadata such as decay model and optional decay-model parameters, as defined for example in .dec decay files).
- A <span style="color:green">decay chain</span> - can be seen as a mother particle and a list of decay modes.

In [ ]:
from decaylanguage.decay.decay import DaughtersDict, DecayMode, DecayChain

**Daughters list** (actually a ``Counter`` dictionary, internally):

In [ ]:
# Constructor from a dictionary
dd = DaughtersDict({'K+': 1, 'K-': 2, 'pi+': 1, 'pi0': 1})

# Constructor from a list of particle names
dd = DaughtersDict(['K+', 'K-', 'K-', 'pi+', 'pi0'])

# Constructor from a string representing the final state
dd = DaughtersDict('K+ K- pi0')
dd

#### Decay Modes

In [ ]:
# A 'default' and hence empty, decay mode
dm = DecayMode()

# Decay mode with minimal input information
dd = DaughtersDict('K+ K-')
dm = DecayMode(0.5, dd)

# Decay mode with decay model information and user metadata
dm = DecayMode(0.2551,                                              # branching fraction
               'pi- pi0 nu_tau',                                    # final-state particles
               model='TAUHADNU',                                    # decay model
               model_params=[-0.108, 0.775, 0.149, 1.364, 0.400],   # decay-model parameters
               study='toy', year=2019                               # user metadata
              )
dm

In [ ]:
print(dm.describe())

Various manipulations are available:

In [ ]:
dm = DecayMode.from_pdgids(0.5, [321, -321])
print(dm)

dm = DecayMode(1.0, 'K+ K+ pi-')
dm.charge_conjugate()

#### Decay chains 

In [ ]:
dm1 = DecayMode(0.0124, 'K_S0 pi0', model='PHSP')
dm2 = DecayMode(0.692, 'pi+ pi-')
dm3 = DecayMode(0.98823, 'gamma gamma')
dc = DecayChain('D0', {'D0':dm1, 'K_S0':dm2, 'pi0':dm3})

dc

In [ ]:
dc.decays

Flatten the decay chain, i.e. replace all intermediate, decaying particles, with their final states:
- The BF is now the *visible BF*

In [ ]:
dc.flatten()

Of course you can sill just as easily visualise decays defined via this `DecayChain` class:

In [ ]:
DecayChainViewer(dc.to_dict())